In [1]:
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori,association_rules
#from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

# Specify the encoding using the 'encoding' parameter
data = pd.read_csv('Online Retail.csv') # Try 'latin-1' first. If it doesn't work, try other encodings like 'cp1252'
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [2]:
data.shape
data.isnull().values.any()
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [3]:
data['Description'] = data['Description'].str.strip()

data.dropna(axis=0,subset=['InvoiceNo'],inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

data = data[~data['InvoiceNo'].str.contains('C')]

data.Country.unique()

basket_France = (data[data['Country']=="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

def hot_encode(x):
  if(x<=0):
    return 0
  if(x>=1):
    return 1

basket_encoded = basket_France.applymap(hot_encode)
basket_France=basket_encoded

basket_France.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
freq_items = apriori(basket_France, min_support=0.1, use_colnames=True)

# Pass the entire freq_items DataFrame, not just the itemsets column
rules = association_rules(freq_items, metric='lift', min_threshold=1)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

print(rules.head())

                                 antecedents                      consequents  \
40           (SET/6 RED SPOTTY PAPER PLATES)    (SET/6 RED SPOTTY PAPER CUPS)   
42  (POSTAGE, SET/6 RED SPOTTY PAPER PLATES)    (SET/6 RED SPOTTY PAPER CUPS)   
35       (STRAWBERRY LUNCH BOX WITH CUTLERY)                        (POSTAGE)   
27      (ROUND SNACK BOXES SET OF4 WOODLAND)                        (POSTAGE)   
41             (SET/6 RED SPOTTY PAPER CUPS)  (SET/6 RED SPOTTY PAPER PLATES)   

    antecedent support  consequent support   support  confidence      lift  \
40            0.127551            0.137755  0.122449    0.960000  6.968889   
42            0.107143            0.137755  0.102041    0.952381  6.913580   
35            0.122449            0.765306  0.114796    0.937500  1.225000   
27            0.158163            0.765306  0.147959    0.935484  1.222366   
41            0.137755            0.127551  0.122449    0.888889  6.968889   

    leverage  conviction  zhangs_metric  
40

In [1]:
pip install mlxtend==0.23.1